In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create dataset -> DataFrame
### **Inference dataset for fashion recommendation**
---
## Define the Dataset Schema (Columns)

In [ ]:
NUMERIC_COLS = [
    "view_count",
    "click_count",
    "purchase_count",
    "length_cm",
    "price",
    "stocks",
]

OBJECT_COLS = [
    "item_id",
    "category",
    "subcategory",
    "sleeve_type",
    "season",
    "fabric",
    "occasion",
    "formality_level",
    "size_range",
    "brand",
]

# Preprocessing data
data = [
    {
        "item_id": "TNC_001",
        "category": "tops",
        "subcategory": "printed_tshirts",
        "sleeve_type": "short",
        "season": "summer",
        "fabric": "cotton",
        "occasion": "casual",
        "formality_level": "low",
        "size_range": "M",
        "brand": "nike",
        "view_count": 1200,
        "click_count": 230,
        "purchase_count": 45,
        "length_cm": 70,
        "price": 199000,
        "stocks": 32,
    },
    {
        "item_id": "TNC_002",
        "category": "tops",
        "subcategory": "printed_hoodies",
        "sleeve_type": "long",
        "season": "winter",
        "fabric": "fleece",
        "occasion": "casual",
        "formality_level": "low",
        "size_range": "L",
        "brand": "adidas",
        "view_count": 800,
        "click_count": 140,
        "purchase_count": 28,
        "length_cm": 74,
        "price": 349000,
        "stocks": 18,
    }
]